# Day 4
## Part 1
I've finally got round to packaging some of the repetitive stuff.

In [1]:
import advent

def parse_data(s):
    g = advent.parse_grid(s)
    return {p for p in g if g[p] == "@"}

def part_1(data):
    return sum(
        1 if len(data & {p + d for d in advent.all_directions()}) < 4 else 0
        for p in data
    )

test_data = parse_data(
    """..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@.""")

assert part_1(test_data) == 13

In [2]:
data = parse_data(advent.read_input())

part_1(data)

1344

## Part 2

In [4]:
def accessible(rolls):
    return {
        p
        for p in rolls
        if len(rolls & {p + d for d in advent.all_directions()}) < 4
    }

def rolls_removed(rolls):
    while (remove := accessible(rolls)):
        yield len(remove)
        rolls = rolls - remove

def part_2(data):
    return sum(rolls_removed(data))

assert part_2(test_data) == 43

In [5]:
part_2(data)

8112

### Appendix

That felt a bit slow.

In [6]:
%%timeit

part_2(data)

1.69 s ± 5.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Try tracking the number of neighbouring rolls and updating when one is removed.

In [17]:
def part_2a(data):
    rolls = {
        p: len(data & {p + d for d in advent.all_directions()})
        for p in data
    }
    result = 0
    while (remove := [p for p in rolls if rolls[p] < 4]):
        for p in remove:
            for nbr in (p + d for d in advent.all_directions() if p + d in rolls):
                rolls[nbr] -= 1
            del rolls[p]
        result += len(remove)
    return result

assert part_2a(test_data) == 43

In [18]:
part_2a(data)

8112

In [19]:
%%timeit

part_2a(data)

171 ms ± 337 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Quite a bit faster. Now try keeping track of which rolls have lost a neighbour.

In [20]:
def part_2b(data):
    rolls = {
        p: len(data & {p + d for d in advent.all_directions()})
        for p in data
    }
    result = 0
    to_check = rolls
    while (remove := [p for p in to_check if rolls.get(p, 5) < 4]):
        to_check = set()
        for p in remove:
            for nbr in (p + d for d in advent.all_directions() if p + d in rolls):
                rolls[nbr] -= 1
                to_check.add(nbr)
            del rolls[p]
        result += len(remove)
    return result

assert part_2a(test_data) == 43

In [21]:
part_2b(data)

8112

In [22]:
%%timeit

part_2b(data)

140 ms ± 681 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Not that much faster.